In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from seaborn import heatmap
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import plotly.express as px
from matplotlib import rcParams
import seaborn as sns
from sklearn.model_selection import KFold
%matplotlib inline

**Data processing:**

In [ ]:
df_2020=pd.read_csv('../input/world-happiness-report/2020.csv')
df_2019=pd.read_csv('../input/world-happiness-report/2019.csv')
df_2018=pd.read_csv('../input/world-happiness-report/2018.csv')
df_2017=pd.read_csv('../input/world-happiness-report/2017.csv')
df_2016=pd.read_csv('../input/world-happiness-report/2016.csv')
df_2015=pd.read_csv('../input/world-happiness-report/2015.csv')
# pick up common features and rename

# print(df_2015.columns)
# print(df_2016.columns)
# print(df_2017.columns)
# print(df_2018.columns)
# print(df_2019.columns)
# print(df_2020.columns)

df_2017 = df_2017.rename(columns = {'Happiness.Score' : 'Happiness Score', 'Happiness.Rank' : 'Happiness Rank', 'Economy..GDP.per.Capita.' : 'Economy (GDP per Capita)', 'Health..Life.Expectancy.' : 'Health (Life Expectancy)','Trust..Government.Corruption.' : 'Trust (Government Corruption)', 'Dystopia.Residual' : 'Dystopia Residual'})
df_2018 = df_2018.rename(columns = {'Healthy life expectancy' : 'Health (Life Expectancy)', 'Social support' : 'Family', 'Score' : 'Happiness Score', 'Country or region':'Country', 'Overall rank' : 'Happiness Rank', 'GDP per capita' : 'Economy (GDP per Capita)', 'Perceptions of corruption' : 'Trust (Government Corruption)'})
df_2019 = df_2019.rename(columns = {'Freedom to make life choices' : 'Freedom', 'Score' : 'Happiness Score', 'Country or region':'Country', 'Overall rank' : 'Happiness Rank', 'GDP per capita' : 'Economy (GDP per Capita)', 'Social support' : 'Family','Healthy life expectancy' : 'Health (Life Expectancy)','Freedom to make life choices' : 'Freedom', 'Perceptions of corruption' : 'Trust (Government Corruption)'})
df_2020 = df_2020.rename(columns = {'Ladder score' : 'Happiness Score', 'Country name':'Country', 'Regional indicator':'Region', 'Explained by: Log GDP per capita' : 'Economy (GDP per Capita)', 'Social support' : 'Family', 'Healthy life expectancy' : 'Health (Life Expectancy)','Freedom to make life choices' : 'Freedom', 'Perceptions of corruption' : 'Trust (Government Corruption)'})
df_2015.drop(["Happiness Rank", "Region", "Standard Error", "Dystopia Residual"], inplace = True, axis = 1)
df_2016.drop(["Dystopia Residual", "Happiness Rank", "Region", "Lower Confidence Interval", "Upper Confidence Interval"], inplace = True, axis = 1)
df_2017.drop(["Happiness Rank", "Dystopia Residual", "Whisker.low", "Whisker.high"], inplace = True, axis = 1)
df_2018.drop(["Freedom to make life choices", "Happiness Rank"], inplace = True, axis = 1)
df_2019.drop("Happiness Rank", inplace = True, axis = 1)
df_2020.drop(["Logged GDP per capita", "Region", "Standard error of ladder score", "upperwhisker", "lowerwhisker", "Ladder score in Dystopia", "Explained by: Social support", "Explained by: Healthy life expectancy", "Explained by: Freedom to make life choices", "Explained by: Generosity", "Explained by: Perceptions of corruption", "Dystopia + residual"], inplace = True, axis = 1)
df_2015['year'] = 2015
df_2016['year'] = 2016
df_2017['year'] = 2017
df_2018['year'] = 2018
df_2019['year'] = 2019
df_2020['year'] = 2020




df_all = pd.concat([df_2019,df_2018,df_2017,df_2016,df_2015])
# fill null values with mean
df_all.fillna((df_all.mean()), inplace = True)
print(df_all.info())
# print(df_all)



**Data Overview**

In [ ]:
plt.plot(df_all.groupby("year").mean()["Happiness Score"])
plt.ylabel("Happiness Score")
plt.ylim(5, 5.6)




In [ ]:
df_top = df_all.head(5).sort_values('Happiness Score', ascending = True)
px.bar(df_top, x='Happiness Score', y='Country', title = "top 10")

KNN

In [ ]:
y = df_all['Happiness Score']
X = df_all[['Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Generosity', 
          'Trust (Government Corruption)']] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
constant_mse = mean_squared_error(np.ones(y_test.shape) * np.mean(y_train), y_test)
print("Constant MSE: %.2f" % constant_mse)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

mse_list = []
k_range = range(1, 51)
for k in k_range:
    model = KNeighborsRegressor(k)
    model.fit(X_train, y_train)
    mse_list.append(mean_squared_error(model.predict(X_test), y_test))
k_list = [k for k in k_range]
plt.plot(k_list, mse_list)
plt.plot(k_list, [constant_mse for i in range(len(k_list))])
plt.show()

# best parameter for knn
knnReg = KNeighborsRegressor (n_neighbors = 4)
knnReg.fit(X_train,y_train)
y_pred = knnReg.predict(X_test).reshape(-1, 1)

score = r2_score(y_test,y_pred)
# print(r2_sc)



**linear regression**

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
lm = LinearRegression()
lm.fit(X_train,y_train)
y_pred = lm.predict(X_test)
# score = lm.score(X_test, y_test)
score = r2_score(y_test, y_pred)
# score = r2_score(y_pred, y_test)
mse = mean_squared_error(y_test, y_pred)
score, mse





In [ ]:
dtr= DecisionTreeRegressor()
dtr.fit(X_train,y_train)
y_pred = dtr.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
y_pred_train = dtr.predict(X_train)
train_mse = mean_squared_error(y_train, y_pred_train)
# d=dtr.score(X_test, y_test)
score = r2_score(y_test, y_pred)
score, test_mse 

In [ ]:
rf = RandomForestRegressor(n_estimators = 13579)
rf.fit(X_train, y_train)
y_hat = rf.predict(X_test)
errors = abs(y_hat - y_test)
acc = 1 - errors
# c=rf.score(X_test, y_test)
score = r2_score(y_test, y_pred)
score, np.mean(acc)